In [1]:
import pandas as pd
import numpy as np

In [15]:
emails = pd.read_csv('DataSets/index/train.csv',sep=" ",header=None,names=['category','paths'],nrows=10000,skiprows=40000)
emails['body'] = ''
emails.groupby("category").size()

category
ham     3926
spam    6074
dtype: int64

In [16]:
import email
import codecs
import re
from bs4 import BeautifulSoup
for index in range(0,len(emails.paths)):
    fp = codecs.open("DataSets"+emails.paths[index][2:], "r",encoding='utf-8', errors='ignore')
    email_text = email.message_from_file(fp)
    texts = ""
    if  email_text.is_multipart():
        for part in email_text.get_payload():
            if part.get_content_maintype() == 'text':
                texts += part.get_payload()
    else:
        texts += email_text.get_payload()
    texts = re.sub('\n','',texts)
    texts = re.sub('_','',texts)
    texts = re.sub('-',' ',texts)
    texts = re.sub('/',' ',texts)
    texts = re.sub(':',' ',texts)
    texts = re.sub('$','',texts)
    texts = re.sub('=','',texts)
    texts = re.sub('<.*?>', '', texts)
    emails.body[index] = " ".join((BeautifulSoup(texts.lower(), 'html.parser').findAll(text=True)))
    fp.close()
emails.head(10)

,category,paths,body
0,spam,../data/inmail.40001,v\ * {behavior url(#default#vml);}o\ * {behavi...
1,ham,../data/inmail.40002,hi!i was wondering if there is a package in r ...
2,ham,../data/inmail.40003,"amanda,this sounds like a pci error of some so..."
3,ham,../data/inmail.40004,# new ticket created by kay uwe huell # pleas...
4,spam,../data/inmail.40005,ladies always giggled at me and even guys did ...
5,ham,../data/inmail.40006,i'm trying to write a script to efficiently ma...
6,spam,../data/inmail.40007,hi.we carry high grade rolex replica watches. ...
7,spam,../data/inmail.40008,everybody knows that a cart1er w4tch is a sile...
8,spam,../data/inmail.40009,documento sin título ...
9,spam,../data/inmail.40010,yours discuss friend ought.safe & effective pe...


In [17]:
temails = emails[emails.body!='']
temails.head(10)

,category,paths,body
0,spam,../data/inmail.40001,v\ * {behavior url(#default#vml);}o\ * {behavi...
1,ham,../data/inmail.40002,hi!i was wondering if there is a package in r ...
2,ham,../data/inmail.40003,"amanda,this sounds like a pci error of some so..."
3,ham,../data/inmail.40004,# new ticket created by kay uwe huell # pleas...
4,spam,../data/inmail.40005,ladies always giggled at me and even guys did ...
5,ham,../data/inmail.40006,i'm trying to write a script to efficiently ma...
6,spam,../data/inmail.40007,hi.we carry high grade rolex replica watches. ...
7,spam,../data/inmail.40008,everybody knows that a cart1er w4tch is a sile...
8,spam,../data/inmail.40009,documento sin título ...
9,spam,../data/inmail.40010,yours discuss friend ought.safe & effective pe...


In [ ]:
import nltk
nltk.download()
from nltk.stem.snowball import SnowballStemmer
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        stemmer = SnowballStemmer("english", ignore_stopwords=True)
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [38]:
#applying bag of words algorithm
#segment each text file into words (for English splitting by space), and 
#count # of times each word occurs in each document and finally assign each word an integer id.
from sklearn.feature_extraction.text import CountVectorizer
count_vect = StemmedCountVectorizer(stop_words='english').fit(temails.body)
X_train_counts = count_vect.transform(temails.body)
X_train_counts.shape

LookupError: 
**********************************************************************
  Resource 'corpora/stopwords' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/home/aditya/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [28]:
# TF-IDF i.e Term Frequency times inverse document frequency.
# TF:  Just cWindesheim Honours College "The Holland Scholarship" Challenge
#      it will give more weightage to longer documents than shorter documents.
# IDF: Finally, we can even reduce the weightage of more common words like (the, is, an etc.) 
#      which occurs in all document.
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
X_train_tfidf.shape

(9033, 155807)

In [29]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, temails.category )
clf.score(X_train_tfidf, temails.category)

0.99269345732314851

In [32]:
text = input("Enter Email body").lower()
X_predict = count_vect.transform(np.array([text]))
X_predict_tfidf = tfidf_transformer.transform(X_predict)
X_predict_tfidf.shape

Enter Email bodyI am really sorry if you find my work unsatisfactory but atleast you can tell me what is unsatisfactory. I will try to improve myself. I am already been working with you like 1 month and after all that it feels so worthless to discontinue inbetween. Sometimes it gets late because there are submissions going on in college but i take these things as seriously and try as much as possible to do tasks assigned to me.


(1, 155807)

In [33]:
clf.predict(X_predict_tfidf)

array(['ham'],
      dtype='<U4')